In [1]:
TABLE = "//home/cvtest/cv-gen-test/akkirr/YAGEN-1210-Relevance/defects_sample_10k"

OUTPUT = "sbs_logs_dedup_rel_random_1k_downloaded_validated2.jsonl"

In [2]:
import yt.type_info.typing 
import yt.wrapper as yt
import pandas as pd
import yt_proxy
from tqdm.auto import tqdm
from pathlib import Path
import cv2
from typing import *
import numpy as np
import hashlib
import jsonlines


client = yt_proxy.yt_proxy._make_client()


yt_path = yt.TablePath(
    name=TABLE,
    client=client,
)

In [3]:
table_iterator = client.read_table(yt_path, raw=False)
table_iterator = tqdm(
    table_iterator,
    desc="Downloading progress",
)


ROOT = Path("../data")
IMAGES_ROOT = ROOT / "images"
TABLE_OUTPUT = ROOT / OUTPUT


def process_image(image: Any, folder):
    image = cv2.imdecode(
        np.frombuffer(image._bytes, dtype=np.uint8), cv2.IMREAD_ANYCOLOR
    )
    hash = hashlib.md5(image.tobytes()).hexdigest()
    filename = hash + ".png"
    path = folder / filename
    cv2.imwrite(str(path), image)
    return filename


for i, row in enumerate(table_iterator):
    out = {}
    out["bad_image_label"] = row.pop("bad_image_label")
    out["good_image_label"] = row.pop("good_image_label")
    out["bad_image"] = process_image(row.pop("bad_image"), IMAGES_ROOT)
    out["good_image"] = process_image(row.pop("good_image"), IMAGES_ROOT)
    with jsonlines.open(TABLE_OUTPUT, 'a') as writer:
        writer.write(out)